In [7]:
!pip install opendatasets
!pip install pandas


In [5]:
{"username":"sanathrajs","key":"550f81eae76c1ae70275c211c8fa73b1"}

{'username': 'sanathrajs', 'key': '550f81eae76c1ae70275c211c8fa73b1'}

In [8]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/mtesconi/twitter-deep-fake-text")


Skipping, found downloaded files in "./twitter-deep-fake-text" (use force=True to force download)


In [9]:
import os

# Path to the dataset
dataset_path = "twitter-deep-fake-text"
# List files in the downloaded directory
files = os.listdir(dataset_path)
print("Files in the dataset directory:", files)


Files in the dataset directory: ['validation.csv', 'test.csv', 'train.csv']


In [10]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, Attention, Concatenate, GlobalAveragePooling1D
from tensorflow.keras import Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Layer
import os

# Download NLTK data if not already done
nltk.download('stopwords')
nltk.download('punkt')

# Step 1: Load Dataset
data = pd.read_csv("train.csv")
print(data.head())

# Step 2: Preprocess Text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+|#\w+', '', text)  # Remove mentions and hashtags
    text = re.sub(r'\W|_', ' ', text)  # Remove special characters, numbers, and punctuations
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_words = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_words)

data['cleaned_tweet'] = data['tweet'].apply(preprocess_text)

# Step 3: Label Encoding
label_mapping = {'human': 1, 'bot': 0}  # Update based on actual dataset
data['class_type'] = data['class_type'].map(label_mapping)

# Step 4: Split data
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_tweet'], data['class_type'], test_size=0.2, random_state=42)

# Step 5: Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_length = 100  # Maximum length of a tweet
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

# Step 6: Load Pre-trained Word Embeddings (e.g., GloVe)
embedding_dim = 100  # Using GloVe with 100 dimensions (you can also use other embedding files)
embedding_index = {}
with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:  # Make sure to have the GloVe file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create embedding matrix for the words in our dataset
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Step 7: Build the Advanced Model with BiLSTM + Attention Mechanism
class AttentionLayer(Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.attention_weights = self.add_weight(name='attention_weights', shape=(input_shape[2],), initializer='zeros')

    def call(self, inputs):
        attention_scores = tf.nn.softmax(inputs @ self.attention_weights)
        weighted_input = inputs * attention_scores
        return tf.reduce_sum(weighted_input, axis=1)

# Define the model
input_layer = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim,
                            weights=[embedding_matrix], input_length=max_length, trainable=False)(input_layer)
bilstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention_layer = AttentionLayer()(bilstm_layer)
dense_layer = Dense(1, activation='sigmoid')(attention_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=dense_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Step 9: Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Step 10: Make predictions
y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")

# Step 11: Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

Files in the dataset directory: ['validation.csv', 'test.csv', 'train.csv']


FileNotFoundError: [Errno 2] No such file or directory: 'twitter-deep-fake-text/tweets.csv'